# Ramp of the simulation

In [1]:
#-----Importa paquetes escenciales

from __future__ import division
import hoomd
import hoomd.md

#-----Define las variables importantes

p_prev = 0.5;
p_new  = 0.7;
t_max  = 5.0;
time_steps = 2.0e5; 

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_5.0_P_0.5_initializing.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integra usando un NPT

pressure = hoomd.variant.linear_interp(points = [(0,p_prev), (time_steps, p_new)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max , tau = 1.5, tauP = 1.5, P = pressure);

#-----Guarda la información de la simulación en un archivo .log y en un .gsd

log_file = "T_" + str(t_max) + "_P_" + str(p_new) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_new) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_new) + "_ramp.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 9000 particles
notice(2): constrain.rigid(): Removing all particles part of rigid bodies (except central particles).Particle tags may change.
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
*

In [ ]:
hoomd.run(time_steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 335252 / 535000 | TPS 25.1758 | ETA 02:12:14
Time 00:00:20 | Step 335521 / 535000 | TPS 26.7773 | ETA 02:04:09
Time 00:00:30 | Step 335804 / 535000 | TPS 28.1499 | ETA 01:57:56
Time 00:00:40 | Step 336095 / 535000 | TPS 28.8988 | ETA 01:54:42
Time 00:00:50 | Step 336381 / 535000 | TPS 28.4073 | ETA 01:56:31
Time 00:01:00 | Step 336661 / 535000 | TPS 27.8084 | ETA 01:58:52
Time 00:01:10 | Step 336937 / 535000 | TPS 27.5614 | ETA 01:59:46
Time 00:01:20 | Step 337207 / 535000 | TPS 26.9345 | ETA 02:02:23
Time 00:01:30 | Step 337466 / 535000 | TPS 25.8121 | ETA 02:07:32
Time 00:01:40 | Step 337737 / 535000 | TPS 26.9885 | ETA 02:01:49
Time 00:01:50 | Step 338003 / 535000 | TPS 26.5222 | ETA 02:03:47
Time 00:02:00 | Step 338274 / 5